# Unit II. Implementing a Predictor from scratch

Universidad Politecnica de Yucatan

Course: Machine Learning

Student: Victor Alejandro Moo Quintal

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron

In [2]:
#Load Data
path = r"C:\Users\cokit\Downloads\Indicadores_municipales_sabana_DA.csv"
df = pd.read_csv(path, encoding='latin-1')
print(df)


      ent         nom_ent  mun  clave_mun                nom_mun  \
0       1  Aguascalientes    1       1001         Aguascalientes   
1       1  Aguascalientes    2       1002               Asientos   
2       1  Aguascalientes    3       1003               Calvillo   
3       1  Aguascalientes    4       1004                  Cosío   
4       1  Aguascalientes    5       1005            Jesús María   
...   ...             ...  ...        ...                    ...   
2451   32       Zacatecas   54      32054          Villa Hidalgo   
2452   32       Zacatecas   55      32055             Villanueva   
2453   32       Zacatecas   56      32056              Zacatecas   
2454   32       Zacatecas   57      32057               Trancoso   
2455   32       Zacatecas   58      32058  Santa María de la Paz   

      pobtot_ajustada    pobreza  pobreza_e  pobreza_m    vul_car  ...  \
0              794304  30.531104   2.264478  28.266627  27.983320  ...   
1               48592  67.111172   

In [3]:
#Data Preprocessing
df = df.drop(columns=['ent', 'nom_ent', 'mun', 'clave_mun', 'nom_mun', 'gdo_rezsoc00'])
df


,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,vul_ing,npnv,ic_rezedu,ic_asalud,ic_segsoc,...,pobreza_alim_10,pobreza_cap_90,pobreza_cap_00,pobreza_cap_10,pobreza_patrim_90,pobreza_patrim_00,pobreza_patrim_10,gini_90,gini_00,gini_10
0,794304,30.531104,2.264478,28.266627,27.983320,8.419106,33.066469,14.970553,24.034493,41.799885,...,11.805700,20.4,12.7,18.474600,43.4,33.7,41.900398,0.473,0.425,0.422628
1,48592,67.111172,8.040704,59.070468,22.439389,5.557604,4.891835,21.222712,15.514032,78.003570,...,21.993299,39.9,29.0,30.980801,64.2,48.9,59.175800,0.379,0.533,0.343879
2,53104,61.360527,7.241238,54.119289,29.428583,2.921336,6.289554,27.361207,20.812551,80.051980,...,19.266800,39.5,33.1,28.259199,63.9,57.9,56.504902,0.414,0.465,0.386781
3,14101,52.800458,4.769001,48.031458,27.128568,7.709276,12.361698,20.889023,14.071657,65.831374,...,14.303200,35.2,21.0,22.386101,59.7,40.1,51.164501,0.392,0.541,0.344984
4,101379,45.338512,6.084037,39.254475,26.262912,8.279864,20.118712,20.578144,16.567818,52.616992,...,15.085100,36.6,22.6,22.139999,60.6,42.2,45.703899,0.391,0.469,0.458083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,21016,74.848837,12.301183,62.547654,19.229856,3.177689,2.743618,27.350040,36.056322,76.550988,...,30.055300,51.8,54.8,41.368999,73.5,70.9,70.859596,0.403,0.589,0.342037
2452,27385,65.450191,10.203506,55.246687,23.623556,5.007426,5.918827,29.914879,53.313420,74.542926,...,13.138800,34.2,25.9,20.563601,57.8,44.1,46.659199,0.422,0.463,0.362527
2453,117528,29.541959,3.535624,26.006335,16.644262,8.828019,44.985759,11.936088,18.316528,32.666426,...,7.164800,15.7,20.7,12.115300,36.6,41.8,32.302700,0.528,0.498,0.436339
2454,20456,78.374962,14.607016,63.767946,13.750759,4.440331,3.433948,26.649950,11.769479,83.235286,...,21.285900,36.2,36.4,30.037100,60.5,54.7,57.394501,0.380,0.483,0.365307


In [4]:

missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0]
columns_with_missing_values = missing_data.index.tolist()
print(missing_data)


cpic_cv                    2
pobtot_00                 14
pobtot_05                  2
porc_pob_15_analfa00      14
porc_pob_15_analfa05       2
porc_pob614_noasiste00    14
porc_pob614_noasiste05     2
porc_pob15_basicainc00    14
porc_pob15_basicainc05     2
porc_pob_snservsal00      14
porc_pob_snservsal05       2
porc_vivpisotierra00      14
porc_vivpisotierra05       2
porc_vivsnsan00           14
porc_vivsnsan05            2
porc_snaguaent00          14
porc_snaguaent05           2
porc_vivsndren00          14
porc_vivsndren05           2
porc_vivsnenergia00       14
porc_vivsnenergia05        2
porc_vivsnlavadora00      14
porc_vivsnlavadora05       2
porc_vivsnrefri00         14
porc_vivsnrefri05          2
irez_soc00                14
irez_soc05                 2
gdo_rezsoc05               2
l_ocupnac00               14
l_ocupnac05                2
p_rez_edu_90               2
p_rez_edu_00               3
p_ser_sal_00               3
p_viv_pisos_90             2
p_viv_pisos_00

In [5]:

non_numeric_columns = df[columns_with_missing_values].select_dtypes(exclude=['float64', 'int64']).columns
ordinal_mapping = {
    'Muy bajo': 1, 
    'Bajo': 2, 
    'Medio': 3, 
    'Alto': 4, 
    'Muy alto': 5
}
df['gdo_rezsoc05'] = df['gdo_rezsoc05'].map(ordinal_mapping)
df['gdo_rezsoc10'] = df['gdo_rezsoc10'].map(ordinal_mapping)
print(df.isnull().sum().sum())


291


In [6]:

imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df))
df_imputed.columns = df.columns
print(df_imputed.isnull().sum().sum()) 
df = df_imputed
df.head()


0


,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,vul_ing,npnv,ic_rezedu,ic_asalud,ic_segsoc,...,pobreza_alim_10,pobreza_cap_90,pobreza_cap_00,pobreza_cap_10,pobreza_patrim_90,pobreza_patrim_00,pobreza_patrim_10,gini_90,gini_00,gini_10
0,794304.0,30.531104,2.264478,28.266627,27.983320,8.419106,33.066469,14.970553,24.034493,41.799885,...,11.805700,20.4,12.7,18.474600,43.4,33.7,41.900398,0.473,0.425,0.422628
1,48592.0,67.111172,8.040704,59.070468,22.439389,5.557604,4.891835,21.222712,15.514032,78.003570,...,21.993299,39.9,29.0,30.980801,64.2,48.9,59.175800,0.379,0.533,0.343879
2,53104.0,61.360527,7.241238,54.119289,29.428583,2.921336,6.289554,27.361207,20.812551,80.051980,...,19.266800,39.5,33.1,28.259199,63.9,57.9,56.504902,0.414,0.465,0.386781
3,14101.0,52.800458,4.769001,48.031458,27.128568,7.709276,12.361698,20.889023,14.071657,65.831374,...,14.303200,35.2,21.0,22.386101,59.7,40.1,51.164501,0.392,0.541,0.344984
4,101379.0,45.338512,6.084037,39.254475,26.262912,8.279864,20.118712,20.578144,16.567818,52.616992,...,15.085100,36.6,22.6,22.139999,60.6,42.2,45.703899,0.391,0.469,0.458083


In [7]:

print(f"Total rows: {len(df)}")
df['label'] = df.apply(
    lambda row: 0 if row['pobreza'] < 50 else (2 if row['pobreza_e'] > row['pobreza_m'] else 1), axis=1
)
print(df['label'].value_counts())


Total rows: 2456
label
1    1519
2     493
0     444
Name: count, dtype: int64


### KNN Scratch


In [8]:
np.random.seed(0)
indices = np.arange(len(df))
np.random.shuffle(indices)
data = df.drop(columns=['label']).values[indices]
labels = df['label'].values[indices]
split_idx = int(0.8 * len(df))  # 80% training, 20% testing
train_data, test_data = data[:split_idx], data[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]


In [9]:
k = 49
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))
def predict(x, train_data, train_labels, k):
    distances = []
    for i in range(len(train_data)):
        distance = euclidean_distance(x, train_data[i])
        distances.append((distance, train_labels[i]))
    distances.sort(key=lambda x: x[0])
    k_nearest = [label for _, label in distances[:k]]
    label_counts = [0, 0, 0]
    for label in k_nearest:
        label_counts[label] += 1
    return np.argmax(label_counts)
correct = 0
for i in range(len(test_data)):
    prediction = predict(test_data[i], train_data, train_labels, k)
    if prediction == test_labels[i]:
        correct += 1
accuracy = correct / len(test_data)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 81.10%


In [10]:
index = 301 
test_sample = test_data[index]
true_label = test_labels[index]
predicted_label = predict(test_sample, train_data, train_labels, k)
print(f"Test Sample Index: {index}")
print(f"Predicted Label: {predicted_label}")
print(f"True Label: {true_label}")


Test Sample Index: 301
Predicted Label: 1
True Label: 0


### Perceptron Scratch


In [12]:
np.random.seed(0)
indices = np.arange(len(df))
np.random.shuffle(indices)
data = df.drop(columns=['label']).values[indices]
labels = df['label'].values[indices]
split_idx = int(0.8 * len(df))
train_data, test_data = data[:split_idx], data[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]


In [13]:
def train_perceptron(data, labels, learning_rate=0.05, n_iters=1000):
    weights = np.zeros(data.shape[1])
    bias = 0
    for _ in range(n_iters):
        for idx in range(len(data)):
            linear_output = np.dot(data[idx], weights) + bias
            y_predicted = 1 if linear_output >= 0 else 0
            error = labels[idx] - y_predicted
            weights += learning_rate * error * data[idx]
            bias += learning_rate * error
    return weights, bias
is_poor_labels = np.array([0 if label == 0 else 1 for label in train_labels])
weights1, bias1 = train_perceptron(train_data, is_poor_labels)
poor_data = train_data[is_poor_labels == 1]
poor_labels = np.array([0 if label == 1 else 1 for label in train_labels[is_poor_labels == 1]])
weights2, bias2 = train_perceptron(poor_data, poor_labels)
def predict(x, weights, bias):
    linear_output = np.dot(x, weights) + bias
    return 1 if linear_output >= 0 else 0
correct = 0
for i in range(len(test_data)):
    is_poor = predict(test_data[i], weights1, bias1)
    if is_poor == 0:
        predicted_label = 0
    else:
        predicted_label = predict(test_data[i], weights2, bias2) + 1
    true_label = test_labels[i]
    if predicted_label == true_label:
        correct += 1
accuracy = correct / len(test_data)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 96.95%


In [14]:
index = 8 
test_sample = test_data[index]
true_label = test_labels[index]
is_poor = predict(test_sample, weights1, bias1)
if is_poor == 0:
    predicted_label = 0
else:
    predicted_label = predict(test_sample, weights2, bias2) + 1
print(f"Test Sample Index: {index}")
print(f"Predicted Label: {predicted_label}")
print(f"True Label: {true_label}")


Test Sample Index: 8
Predicted Label: 1
True Label: 1


### KNN Libraries


In [16]:
X = df.drop(columns=['label']).values
y = df['label'].values
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=0)
k = 49
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(train_data, train_labels)
predicted_labels = knn.predict(test_data)
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 83.54%


In [17]:
index = 2 
test_sample = np.array([test_data[index]])
true_label = test_labels[index]
predicted_label = knn.predict(test_sample)[0]
print(f"Test Sample Index: {index}")
print(f"Predicted Label: {predicted_label}")
print(f"True Label: {true_label}")


Test Sample Index: 2
Predicted Label: 1
True Label: 1


### Perceptron Libraries


In [18]:
is_poor_labels = np.array([0 if label == 0 else 1 for label in train_labels])
perceptron1 = Perceptron()
perceptron1.fit(train_data, is_poor_labels)


Perceptron()

In [19]:
poor_data = train_data[is_poor_labels == 1]
poor_labels = np.array([0 if label == 1 else 1 for label in train_labels[is_poor_labels == 1]])
perceptron2 = Perceptron()
perceptron2.fit(poor_data, poor_labels)


Perceptron()

In [20]:
predictions_stage1 = perceptron1.predict(test_data)
predictions_stage2 = perceptron2.predict(test_data[predictions_stage1 == 1]) + 1
final_predictions = predictions_stage1.copy()
final_predictions[predictions_stage1 == 1] = predictions_stage2
accuracy = accuracy_score(test_labels, final_predictions)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 95.73%


In [21]:
index = 2 
test_sample = test_data[index].reshape(1, -1)
true_label = test_labels[index]
predicted_stage1 = perceptron1.predict(test_sample)
if predicted_stage1 == 0:
    predicted_label = 0
else:
    predicted_label = perceptron2.predict(test_sample) + 1
print(f"Test Sample Index: {index}")
print(f"Predicted Label: {predicted_label[0]}")
print(f"True Label: {true_label}")


Test Sample Index: 2
Predicted Label: 1
True Label: 1
